In [2]:
import re
import nltk
from bs4 import BeautifulSoup
import numpy as np
from urllib.request import urlopen
import string
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from gensim import matutils

import pyLDAvis.gensim

In [3]:
wiki = "http://en.wikipedia.org/wiki/"
titles = ["Integral", "Riemann_integral", "Riemann-Stieltjes_integral", "Derivative",
    "Limit_of_a_sequence", "Edvard_Munch", "Vincent_van_Gogh", "Jan_Matejko",
    "Lev_Tolstoj", "Franz_Kafka", "J._R._R._Tolkien","Warsaw","England","Eiffel_Tower","Taipei_101"]

def parse(url):        
    x = urlopen(url)
    x = x.read()
    x = BeautifulSoup(x,'html.parser')
    x = x.find("div",id="bodyContent")
    x = x.find_all("p")
    return(x)

articles_raw = [parse(url) for url in [wiki+x for x in titles]]

In [4]:
articles_raw[0][1]

<p>is defined informally as the signed <a class="mw-redirect" href="/wiki/Area_(geometry)" title="Area (geometry)">area</a> of the region in the <span class="texhtml mvar" style="font-style:italic;">xy</span>-plane that is bounded by the <a href="/wiki/Graph_of_a_function" title="Graph of a function">graph</a> of <span class="texhtml mvar" style="font-style:italic;">f</span>, the <span class="texhtml mvar" style="font-style:italic;">x</span>-axis and the vertical lines <span class="texhtml"><i>x</i> = <i>a</i></span> and <span class="texhtml"><i>x</i> = <i>b</i></span>. The area above the <span class="texhtml mvar" style="font-style:italic;">x</span>-axis adds to the total and that below the <span class="texhtml mvar" style="font-style:italic;">x</span>-axis subtracts from the total.
</p>

## Preprocessing

### Removing html tags

In [5]:
articles_non_tags = [re.sub('<.*?>','',str(a)) for a in articles_raw]

### Convert into lower letters 

In [6]:
articles_lower = [art.lower() for art in articles_non_tags]

In [7]:
articles_lower[1][:1000]

'[in the branch of mathematics known as real analysis, the riemann integral, created by bernhard riemann, was the first rigorous definition of the integral of a function on an interval.  it was presented to the faculty at the university of göttingen in 1854, but not published in a journal until 1868.[1] for many functions and practical applications, the riemann integral can be evaluated by the fundamental theorem of calculus or approximated by numerical integration.\n, the riemann integral is unsuitable for many theoretical purposes. some of the technical deficiencies in riemann integration can be remedied with the riemann–stieltjes integral, and most disappear with the lebesgue integral.\n, let f be a nonnegative real-valued function on the interval [a, b], and let\n, be the region of the plane under the graph of the function f and above the interval [a, b] (see the figure on the top right). we are interested in measuring the area of s. once we have measured it, we will denote the are

## Segmentation

### Word tokenize - document segmentation

In [18]:
nltk.word_tokenize(articles_lower[0])[:10]

['[',
 'in',
 'mathematics',
 ',',
 'an',
 'integral',
 'assigns',
 'numbers',
 'to',
 'functions']

In [19]:
article_tokenized = [nltk.word_tokenize(art) for art in articles_lower]

In [20]:
article_tokenized[0][:10]

['[',
 'in',
 'mathematics',
 ',',
 'an',
 'integral',
 'assigns',
 'numbers',
 'to',
 'functions']

## Cleaning segments

### Removing punctuation

In [21]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
articles_no_punctuation = [
    [token for token in art if token not in string.punctuation]
    for art in article_tokenized
]

In [23]:
articles_no_punctuation[5][:10]

['edvard',
 'munch',
 '/mʊŋk/',
 '1',
 'norwegian',
 'ˈedvɑʈ',
 'ˈmuŋk',
 'listen',
 '12',
 'december']

### Removing 'stopwords'

In [24]:
stopwords = nltk.corpus.stopwords.words('english')

In [25]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [26]:
len(stopwords)

179

In [27]:
articles_no_stopwords = [
    [token for token in art if not token in stopwords]
    for art in articles_no_punctuation
]

In [28]:
articles_no_stopwords[0][:10]

['mathematics',
 'integral',
 'assigns',
 'numbers',
 'functions',
 'way',
 'describe',
 'displacement',
 'area',
 'volume']

## Segments normalization

### Stemming - identifying a common stem among various forms of a word (removing suffixes, plural from, various form fo verbs, stem is not required to be a propery spelled word)
### Most popular : Porter and Snowball stemmers

### More sxtensive normalization down to the semmantic root of a word is called lemmatization, more accurate than stemming as it takes into account the meaning of a word, but stemmers are faster, and less complex

W nltk mamy dostępny stemmer dla języka angielskiego. Dla języka polskiego mamy dostępny na przykład stemmer Morfeusz (http://sgjp.pl/morfeusz/index.html),

In [29]:
stemmer = nltk.PorterStemmer()

In [30]:
stemmer.stem('driving')

'drive'

In [31]:
articles_stemmed = [
    [stemmer.stem(token) for token in art]
    for art in articles_no_stopwords
]

In [32]:
articles_stemmed[0][:10]

['mathemat',
 'integr',
 'assign',
 'number',
 'function',
 'way',
 'describ',
 'displac',
 'area',
 'volum']

### Merging into one text

In [33]:
articles_cleaned = [' '.join(tokens) for tokens in articles_stemmed]

In [34]:
articles_cleaned[0][:400]

'mathemat integr assign number function way describ displac area volum concept aris combin infinitesim data integr one two main oper calculu invers oper differenti given function f real variabl x interv b real line definit integr defin inform sign area region xy-plan bound graph f x-axi vertic line x x b. area x-axi add total x-axi subtract total oper integr addit constant invers oper differenti re'

### Count vectorizer (reguires text not separate tokens) - represent documents as vectors of word counts

In [35]:
vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',min_df=2, max_df=0.7) 
# default pattern remove one letter long words

In [36]:
vectorizer.fit(articles_cleaned)
dtm_count = vectorizer.transform(articles_cleaned)
dtm_count

<15x3711 sparse matrix of type '<class 'numpy.int64'>'
	with 14572 stored elements in Compressed Sparse Row format>

In [37]:
dtm_count.shape

(15, 3711)

In [38]:
dtm_count.getcol(800).toarray()

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [2],
       [0],
       [0]])

In [39]:
vectorizer.get_feature_names()[800]

'birthplac'

In [42]:
vectorizer.vocabulary_['mathemat']

2260

### statistics for  word 'kingdom' in documents

In [43]:
vectorizer.vocabulary_["kingdom"]

2089

In [44]:
vectorizer.get_feature_names()[2088]

'king'

In [45]:
dtm_count.getcol(vectorizer.vocabulary_["kingdom"]).toarray()

array([[ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 0],
       [ 0],
       [ 7],
       [ 2],
       [ 3],
       [ 5],
       [56],
       [ 0],
       [ 0]])

### statistics of words in first document

In [46]:
dtm_count.getrow(0).toarray()[0].shape

(3711,)

In [47]:
df = pd.DataFrame({
    'counts':dtm_count.getrow(0).toarray()[0],# pandas potrzebuje 1d
    'words':vectorizer.get_feature_names()
}
)

In [48]:
df = df.sort_values(by="counts",ascending=False)

In [49]:
df.head()

,counts,words
1992,267,integr
1703,132,function
1554,74,f
3677,71,x
731,50,b


### Top k words for all documents

In [50]:
def top_words(matrix,vectorizer,k):
    
    words = np.array(vectorizer.get_feature_names())
    top_words = []
    
    for art in range(matrix.shape[0]):

        words_counter =  matrix.getrow(art).toarray()[0]

        words_counter = np.argsort(words_counter)
        words_counter = words_counter[::-1]
        
        top_words_indices = words_counter[:k]
        top_document_words = words[top_words_indices]
        
        
        top_words.append(top_document_words) 
        
    return np.array(top_words)

    
        

In [51]:
top_words(dtm_count,vectorizer,5)

array([['integr', 'function', 'f', 'x', 'b'],
       ['integr', 'riemann', 'sum', 'interv', 'function'],
       ['integr', 'g', 'riemann', 'stieltj', 'function'],
       ['deriv', 'f', 'function', 'x', 'h'],
       ['displaystyl', 'x', 'sequenc', 'b', 'to'],
       ['cm', 'paint', 'museum', 'art', 'portrait'],
       ['van', 'gogh', 'paint', 'art', 'portrait'],
       ['paint', 'kraków', 'polish', 'jan', '26'],
       ['war', 'russian', 'anarchist', 'novel', 'life'],
       ['stori', 'german', 'der', 'write', 'die'],
       ['tolkien', 'ring', 'english', 'lord', 'stori'],
       ['warsaw', 'citi', 'polish', 'poland', 'palac'],
       ['england', 'english', 'world', 'kingdom', 'london'],
       ['tower', 'level', 'top', 'pari', 'de'],
       ['101', 'floor', 'build', 'tower', 'ft']], dtype='<U14')

### TF-IDF - Term Frequency-Inverse Documents Frequency

In [52]:
sentences = [
    "morze plaza slonce",
    'morze piasek slonce lato',
    'plaza lato woda'
]

In [53]:
CountVectorizer().fit_transform(sentences).toarray()

array([[0, 1, 0, 1, 1, 0],
       [1, 1, 1, 0, 1, 0],
       [1, 0, 0, 1, 0, 1]], dtype=int64)

In [54]:
TfidfVectorizer(norm=None).fit_transform(sentences).toarray()

array([[0.        , 1.28768207, 0.        , 1.28768207, 1.28768207,
        0.        ],
       [1.28768207, 1.28768207, 1.69314718, 0.        , 1.28768207,
        0.        ],
       [1.28768207, 0.        , 0.        , 1.28768207, 0.        ,
        1.69314718]])

In [55]:
# im częściej występuje slowo w danym dokumencie tym to słowo ma większe TF-IDF
# im częściej słowo występuje w wielu artykułąch tym to słowo ma mniejsze TF-IDF

In [56]:
tf_idf = TfidfVectorizer(norm=None,min_df=2,max_df=0.7,token_pattern='(?u)\\b\\w+\\b')

In [57]:
tf_idf.fit(articles_cleaned)
dtm_tfidf = tf_idf.transform(articles_cleaned)
dtm_tfidf

<15x3711 sparse matrix of type '<class 'numpy.float64'>'
	with 14572 stored elements in Compressed Sparse Row format>

In [58]:
top_words(dtm_tfidf,tf_idf,5)

array([['integr', 'function', 'x', 'f', 'interv'],
       ['riemann', 'integr', 'sum', 'interv', 'partit'],
       ['integr', 'riemann', 'stieltj', 'g', 'x'],
       ['f', 'deriv', 'x', 'function', 'vector'],
       ['displaystyl', 'x', 'sequenc', 'b', 'to'],
       ['cm', 'paint', 'museum', 'portrait', 'scream'],
       ['gogh', 'van', 'paint', 'gauguin', '1888'],
       ['kraków', 'polish', 'paint', 'jan', 'fine'],
       ['anarchist', 'war', 'russian', 'novel', 'peac'],
       ['stori', 'der', 'franz', 'jewish', 'czech'],
       ['tolkien', 'ring', 'lord', 'english', 'beowulf'],
       ['warsaw', 'citi', 'polish', 'poland', 'palac'],
       ['england', 'english', 'london', 'kingdom', 'britain'],
       ['tower', 'level', 'top', 'pari', 'ft'],
       ['floor', '101', 'tower', 'build', 'ft']], dtype='<U14')

In [59]:
top_words(dtm_count,vectorizer,5)

array([['integr', 'function', 'f', 'x', 'b'],
       ['integr', 'riemann', 'sum', 'interv', 'function'],
       ['integr', 'g', 'riemann', 'stieltj', 'function'],
       ['deriv', 'f', 'function', 'x', 'h'],
       ['displaystyl', 'x', 'sequenc', 'b', 'to'],
       ['cm', 'paint', 'museum', 'art', 'portrait'],
       ['van', 'gogh', 'paint', 'art', 'portrait'],
       ['paint', 'kraków', 'polish', 'jan', '26'],
       ['war', 'russian', 'anarchist', 'novel', 'life'],
       ['stori', 'german', 'der', 'write', 'die'],
       ['tolkien', 'ring', 'english', 'lord', 'stori'],
       ['warsaw', 'citi', 'polish', 'poland', 'palac'],
       ['england', 'english', 'world', 'kingdom', 'london'],
       ['tower', 'level', 'top', 'pari', 'de'],
       ['101', 'floor', 'build', 'tower', 'ft']], dtype='<U14')

### Cosine similarity 

In [60]:
sentences = ["morze plaza slonce","morze plaza slonce morze plaza slonce"]

In [61]:
matrix_repr = CountVectorizer().fit_transform(sentences).toarray()

In [62]:
euclidean(matrix_repr[0],matrix_repr[1])

1.7320508075688772

In [63]:
cosine(matrix_repr[0],matrix_repr[1])

0.0

In [64]:
dtm_count.shape

(15, 3711)

In [65]:
cosine(dtm_count[0].toarray(),dtm_count[1].toarray())

0.24659537669553977

### according to CountVectorizer

In [66]:
for art in range(dtm_count.shape[0]):
    distances = [cosine(dtm_count[art].toarray(),dtm_count[art2].toarray())for art2 in range(dtm_count.shape[0])]
    distances[art] = 1.0
    print('Article {} is most similar to {}'.format(titles[art],titles[np.argmin(distances)]))

Article Integral is most similar to Riemann_integral
Article Riemann_integral is most similar to Integral
Article Riemann-Stieltjes_integral is most similar to Riemann_integral
Article Derivative is most similar to Integral
Article Limit_of_a_sequence is most similar to Derivative
Article Edvard_Munch is most similar to Jan_Matejko
Article Vincent_van_Gogh is most similar to Edvard_Munch
Article Jan_Matejko is most similar to Edvard_Munch
Article Lev_Tolstoj is most similar to Franz_Kafka
Article Franz_Kafka is most similar to Lev_Tolstoj
Article J._R._R._Tolkien is most similar to Franz_Kafka
Article Warsaw is most similar to Jan_Matejko
Article England is most similar to J._R._R._Tolkien
Article Eiffel_Tower is most similar to Taipei_101
Article Taipei_101 is most similar to Eiffel_Tower


### according to TF-IDF

In [67]:
for art in range(dtm_tfidf.shape[0]):
    distances = [cosine(dtm_tfidf[art].toarray(),dtm_tfidf[art2].toarray())for art2 in range(dtm_tfidf.shape[0])]
    distances[art] = 1.0
    print('Article {} is most similar to {}'.format(titles[art],titles[np.argmin(distances)]))

Article Integral is most similar to Riemann_integral
Article Riemann_integral is most similar to Riemann-Stieltjes_integral
Article Riemann-Stieltjes_integral is most similar to Riemann_integral
Article Derivative is most similar to Integral
Article Limit_of_a_sequence is most similar to Derivative
Article Edvard_Munch is most similar to Jan_Matejko
Article Vincent_van_Gogh is most similar to Edvard_Munch
Article Jan_Matejko is most similar to Edvard_Munch
Article Lev_Tolstoj is most similar to Franz_Kafka
Article Franz_Kafka is most similar to Lev_Tolstoj
Article J._R._R._Tolkien is most similar to Franz_Kafka
Article Warsaw is most similar to Jan_Matejko
Article England is most similar to Lev_Tolstoj
Article Eiffel_Tower is most similar to Taipei_101
Article Taipei_101 is most similar to Eiffel_Tower


## LSI - Latent Semantic Indexing, LSA - Latent Semantic Analysis (SVD)

In [68]:
dtm_tfidf.shape

(15, 3711)

In [69]:
# 15 documents and 3724 terms - OVERFITTTING
# We want to end up with fewer topics than words, so we can use those topic vector 
# as a reduced-dimension reprezentation of the original TF-IDF vectors

In [70]:
svd = TruncatedSVD(5,n_iter=100) # 5 topics

In [71]:
svd.fit(dtm_tfidf)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=100,
       random_state=None, tol=0.0)

In [72]:
svd_topic_vectors = svd.transform(dtm_tfidf)#transform document_term_matrix 
                                            #into topic_document matrix(topic vectors)

In [73]:
svd_topic_vectors.shape

(15, 5)

In [77]:
svd_topic_vectors

array([[ 122.34006,  579.64478,   61.19412,  -50.45245,  -22.24377],
       [  63.85942,  323.87726,   35.0873 ,  -28.10853,  -19.64049],
       [  21.26956,  108.0415 ,   11.54142,   -9.61745,   -5.63918],
       [  77.89685,  342.31017,   38.24582,  -27.26122,    3.21529],
       [  13.97334,   62.15429,    6.63352,   -5.28584,   -0.60542],
       [  65.30551,   -6.67192,  -46.63602,  -41.00558,    8.3564 ],
       [ 328.47639,  -57.06762, -459.70302, -519.80318,  -39.10851],
       [  31.02738,   -2.23506,  -25.19203,    4.68742,   -1.94244],
       [  36.50551,   -2.46899,   -2.94982,    0.06929,   14.95896],
       [  69.57849,   -3.83023,   -2.65704,    0.24222,   20.08586],
       [ 605.71004, -173.30739,  482.57372,  -76.60586, -128.60102],
       [ 382.96687,   12.20754, -371.09711,  528.12232, -161.26747],
       [ 276.79253,   -7.039  ,  -22.16649,   97.25957,  546.59481],
       [  53.9151 ,    5.5439 ,  -25.92881,    6.00826,   33.03741],
       [  36.89758,    3.46018,  -

In [75]:
np.set_printoptions(5, suppress=True)

In [76]:
cumsum = np.cumsum(svd.explained_variance_ratio_)
cumsum # it shows us how each topic explains the variance of our documnets in our new topic vectos space 

array([0.13518, 0.3002 , 0.48821, 0.67127, 0.78089])

In [78]:
svd.singular_values_

array([859.9357 , 779.2308 , 769.9104 , 755.36761, 588.29256])

In [79]:
svd_topic_vectors.shape

(15, 5)

In [80]:
svd.components_.shape # (topics,terms)

(5, 3711)

In [81]:
#Top words in each topic
k=8
words = np.array(vectorizer.get_feature_names())
for topic in svd.components_:
    indexes = np.argsort(topic)
    indexes = indexes[::-1]
    top_words_indices = indexes[:k]
    print(words[top_words_indices])
    print(topic[indexes[:k]])
# each topic is a linear combiantations of terms

['tolkien' 'warsaw' 'gogh' 'van' 'england' 'english' 'citi' 'paint']
[0.59017 0.3205  0.2251  0.19539 0.17532 0.12593 0.11799 0.10793]
['integr' 'function' 'f' 'x' 'riemann' 'deriv' 'interv' 'sum']
[0.60331 0.32601 0.30717 0.26743 0.22145 0.15633 0.13712 0.11995]
['tolkien' 'ring' 'lord' 'integr' 'beowulf' 'english' 'stori' 'fantasi']
[0.58549 0.0959  0.09478 0.067   0.04997 0.04915 0.04287 0.037  ]
['warsaw' 'citi' 'polish' 'poland' 'palac' 'park' 'england' 'centr']
[0.5721  0.19115 0.1223  0.12164 0.095   0.07486 0.06243 0.05259]
['england' 'english' 'london' 'kingdom' 'britain' 'world' 'british'
 'roman']
[0.62098 0.2391  0.16677 0.15607 0.14259 0.11145 0.11074 0.08964]


In [82]:
# topics in documents
svd_topic_vectors = svd_topic_vectors/svd.singular_values_#(documents,topics)
svd_topic_vectors

array([[ 0.14227,  0.74387,  0.07948, -0.06679, -0.03781],
       [ 0.07426,  0.41564,  0.04557, -0.03721, -0.03339],
       [ 0.02473,  0.13865,  0.01499, -0.01273, -0.00959],
       [ 0.09058,  0.43929,  0.04968, -0.03609,  0.00547],
       [ 0.01625,  0.07976,  0.00862, -0.007  , -0.00103],
       [ 0.07594, -0.00856, -0.06057, -0.05429,  0.0142 ],
       [ 0.38198, -0.07324, -0.59709, -0.68815, -0.06648],
       [ 0.03608, -0.00287, -0.03272,  0.00621, -0.0033 ],
       [ 0.04245, -0.00317, -0.00383,  0.00009,  0.02543],
       [ 0.08091, -0.00492, -0.00345,  0.00032,  0.03414],
       [ 0.70437, -0.22241,  0.62679, -0.10142, -0.2186 ],
       [ 0.44534,  0.01567, -0.482  ,  0.69916, -0.27413],
       [ 0.32188, -0.00903, -0.02879,  0.12876,  0.92912],
       [ 0.0627 ,  0.00711, -0.03368,  0.00795,  0.05616],
       [ 0.04291,  0.00444, -0.02012,  0.01896,  0.03921]])

In [83]:
pd.DataFrame(svd_topic_vectors,
             index=titles,
             columns=['topic{}'.format(i) for i in range(svd_topic_vectors.shape[1])])

,topic0,topic1,topic2,topic3,topic4
Integral,0.142267,0.743868,0.079482,-0.066792,-0.037811
Riemann_integral,0.074261,0.415637,0.045573,-0.037212,-0.033386
Riemann-Stieltjes_integral,0.024734,0.138651,0.014991,-0.012732,-0.009586
Derivative,0.090585,0.439292,0.049676,-0.036090,0.005465
Limit_of_a_sequence,0.016249,0.079764,0.008616,-0.006998,-0.001029
Edvard_Munch,0.075942,-0.008562,-0.060573,-0.054286,0.014205
Vincent_van_Gogh,0.381978,-0.073236,-0.597086,-0.688146,-0.066478
Jan_Matejko,0.036081,-0.002868,-0.032721,0.006205,-0.003302
Lev_Tolstoj,0.042451,-0.003168,-0.003831,0.000092,0.025428
Franz_Kafka,0.080911,-0.004915,-0.003451,0.000321,0.034143


In [84]:
for col in range(svd_topic_vectors.shape[1]):
    indexes = np.argsort(svd_topic_vectors[:,col])
    indexes = indexes[::-1]
    print('topic_{}'.format(col))
    print([titles[ind] for ind in indexes])

topic_0
['J._R._R._Tolkien', 'Warsaw', 'Vincent_van_Gogh', 'England', 'Integral', 'Derivative', 'Franz_Kafka', 'Edvard_Munch', 'Riemann_integral', 'Eiffel_Tower', 'Taipei_101', 'Lev_Tolstoj', 'Jan_Matejko', 'Riemann-Stieltjes_integral', 'Limit_of_a_sequence']
topic_1
['Integral', 'Derivative', 'Riemann_integral', 'Riemann-Stieltjes_integral', 'Limit_of_a_sequence', 'Warsaw', 'Eiffel_Tower', 'Taipei_101', 'Jan_Matejko', 'Lev_Tolstoj', 'Franz_Kafka', 'Edvard_Munch', 'England', 'Vincent_van_Gogh', 'J._R._R._Tolkien']
topic_2
['J._R._R._Tolkien', 'Integral', 'Derivative', 'Riemann_integral', 'Riemann-Stieltjes_integral', 'Limit_of_a_sequence', 'Franz_Kafka', 'Lev_Tolstoj', 'Taipei_101', 'England', 'Jan_Matejko', 'Eiffel_Tower', 'Edvard_Munch', 'Warsaw', 'Vincent_van_Gogh']
topic_3
['Warsaw', 'England', 'Taipei_101', 'Eiffel_Tower', 'Jan_Matejko', 'Franz_Kafka', 'Lev_Tolstoj', 'Limit_of_a_sequence', 'Riemann-Stieltjes_integral', 'Derivative', 'Riemann_integral', 'Edvard_Munch', 'Integral', 

In [85]:
# documents' similarity according to topics 
for art_ind in range(svd_topic_vectors.shape[0]):
    distances = [cosine(svd_topic_vectors[art_ind,:],art2) for art2 in svd_topic_vectors]
    distances[art_ind] = 1.0
    print('Article {} is most similar to {}'.format(titles[art_ind],titles[np.argmin(distances)]))

Article Integral is most similar to Riemann-Stieltjes_integral
Article Riemann_integral is most similar to Riemann-Stieltjes_integral
Article Riemann-Stieltjes_integral is most similar to Riemann_integral
Article Derivative is most similar to Limit_of_a_sequence
Article Limit_of_a_sequence is most similar to Derivative
Article Edvard_Munch is most similar to Vincent_van_Gogh
Article Vincent_van_Gogh is most similar to Edvard_Munch
Article Jan_Matejko is most similar to Edvard_Munch
Article Lev_Tolstoj is most similar to Franz_Kafka
Article Franz_Kafka is most similar to Lev_Tolstoj
Article J._R._R._Tolkien is most similar to Franz_Kafka
Article Warsaw is most similar to Jan_Matejko
Article England is most similar to Taipei_101
Article Eiffel_Tower is most similar to Taipei_101
Article Taipei_101 is most similar to Eiffel_Tower


## LDA (Latent Dirichlet Allocation) 
Motywacja: przedstawienie tekstu jako mieszanki tematów.
Temat - rozkład prawdopodobieństwa na zbiorze słów.

**Rozkład Dirichleta**
Jest to rozkład, na którym opiera się model LDA

Gęstość trójwymiarowego rozkładu Dirichleta Dir( αα ).

Wektor losowy  (x1,...,xK)(x1,...,xK)  z  KK -wymiarowego rozkładu Dirichleta to punkt na  (K−1)(K−1) -wymiarowym "trójkącie" (sympleksie), czyli  x1+...+xK=1x1+...+xK=1 ,  xi≥0xi≥0 .

<img src="Grafika/3d_simplex.png" width="500">

<img src="Grafika/Smoothed_LDA.png">


,gdzie

$\theta_d \sim Dir(\alpha)$  - rozkład tematów w dokumencie

$Z \sim Discr(\theta)$ - temat, którego pochodz słowo

$W \sim Discr(\phi_Z)$ - słowo

$\phi_i \sim Dir(\beta)$ - tematy

Dokumenty będą składać sie tylko z kilku tematów (alfa będzie mała).

Tematy będą charakteryzowane również tylko przez cześć słów (beta małe). Dzięki temu możemy ludzkim okiem rozróżnić i zintepretować tematy.

### LDA w module GENSIM
gensim to modul stworzony do analizy tekstów: https://radimrehurek.com/gensim/

### LDA assumes that each document is a mixture (linear combination) of some arbitrary number of topics. LDA also assumes taht each topic can be represented by a distribution of words

### Create corpus from text

In [86]:
dictionary = corpora.Dictionary(articles_stemmed)# create a dictionary from all words

In [87]:
dictionary

In [88]:
dictionary.keys()[-10:]

[10509, 10510, 10511, 10512, 10513, 10514, 10515, 10516, 10517, 10518]

In [89]:
dictionary.get(300)

'determin'

In [90]:
dictionary.id2token[300]

'determin'

In [91]:
dictionary.doc2bow(['mathemat' ,'mathemat','mathemat'])# word nr 644 apears 2 times in this doc

[(626, 3)]

In [92]:
corpus = [dictionary.doc2bow(art) for art in articles_stemmed] # create corpus

In [93]:
len(corpus) # corpus made of all words from all documents

15

In [94]:
len(dictionary) #number of tokens in dictionary 

10519

In [95]:
len(corpus[0])# number of words in first document

1084

In [98]:
corpus[0][:5] # word nb 0 epears 18 times

[(0, 18), (1, 1), (2, 15), (3, 1), (4, 1)]

In [99]:
model_corpus = LdaModel(corpus=corpus,id2word=dictionary,num_topics=5)

### Create corpus from sparse matrix

In [100]:
vectorizer = CountVectorizer(min_df=0.2,max_df=0.5)
dtm_cv = vectorizer.fit_transform(articles_cleaned)

In [101]:
corpus_sparse = matutils.Sparse2Corpus(dtm_cv,documents_columns=False) # by default documents in columns

In [102]:
dictionary_sparse = vectorizer.get_feature_names()

In [103]:
len(dictionary_sparse)

1909

In [104]:
dictionary_sparse_dict = {i:j for i,j in enumerate(dictionary_sparse)}

In [105]:
dictionary_sparse_dict[615]

'copi'

In [106]:
dictionary_sparse = corpora.Dictionary.from_corpus(corpus=corpus_sparse,id2word=dictionary_sparse_dict)

In [107]:
model_corpus_sparse = LdaModel(corpus=corpus_sparse,id2word=dictionary_sparse,num_topics=5)

### LDA model

In [108]:
model_corpus = LdaModel(corpus=corpus_sparse,id2word=dictionary_sparse,num_topics=5, alpha='auto',eta='auto')

In [109]:
model_corpus.get_document_topics(corpus_sparse[5],minimum_probability=0)

[(0, 0.8084837),
 (1, 0.000386054),
 (2, 0.19042878),
 (3, 0.00035596616),
 (4, 0.0003454574)]

In [110]:
for i in range(len(corpus_sparse)):
    print(model_corpus.get_document_topics(corpus_sparse[i],minimum_probability=0))

[(0, 0.00024527204), (1, 0.00029643162), (2, 0.00025273184), (3, 0.9989192), (4, 0.00028634755)]
[(0, 0.00037598817), (1, 0.00042959556), (2, 0.0003659157), (3, 0.99841464), (4, 0.00041390446)]
[(0, 0.001335877), (1, 0.0014931974), (2, 0.0012785523), (3, 0.9944392), (4, 0.0014531367)]
[(0, 0.0005166747), (1, 0.9952794), (2, 0.0005007746), (3, 0.0031121871), (4, 0.00059098087)]
[(0, 0.9730465), (1, 0.0017814501), (2, 0.0013653358), (3, 0.011274056), (4, 0.012532664)]
[(0, 0.8202629), (1, 0.00038635463), (2, 0.1786489), (3, 0.00035624887), (4, 0.0003455638)]
[(0, 0.5058793), (1, 0.00032987475), (2, 0.49346766), (3, 0.00016512933), (4, 0.00015800908)]
[(0, 0.9956163), (1, 0.0013076743), (2, 0.0009025927), (3, 0.001123489), (4, 0.001049956)]
[(0, 0.076942496), (1, 0.0005022746), (2, 0.00041414777), (3, 0.92163783), (4, 0.00050325936)]
[(0, 0.855582), (1, 0.004198087), (2, 0.13826728), (3, 0.00083091133), (4, 0.0011217478)]
[(0, 0.9727455), (1, 0.0053994074), (2, 0.010167487), (3, 0.0045636

In [111]:
titles

['Integral',
 'Riemann_integral',
 'Riemann-Stieltjes_integral',
 'Derivative',
 'Limit_of_a_sequence',
 'Edvard_Munch',
 'Vincent_van_Gogh',
 'Jan_Matejko',
 'Lev_Tolstoj',
 'Franz_Kafka',
 'J._R._R._Tolkien',
 'Warsaw',
 'England',
 'Eiffel_Tower',
 'Taipei_101']

In [117]:
for tup in model_corpus.get_topic_terms(topicid=1):
    print(dictionary_sparse[tup[0]],tup[1])

warsaw 0.021059845
tower 0.015536538
integr 0.011015256
van 0.008835143
gogh 0.007441115
level 0.0059618913
england 0.005725723
polish 0.005702318
poland 0.0050903345
riemann 0.004766491


In [118]:
model_corpus.alpha

array([0.34437, 0.36998, 0.32482, 0.36194, 0.35779], dtype=float32)

In [119]:
model_corpus.eta

array([0.34243, 0.41751, 0.33211, ..., 0.34431, 0.32807, 0.40668],
      dtype=float32)

In [120]:
model_corpus.print_topics(num_words=8)

[(0,
  '0.016*"england" + 0.013*"gogh" + 0.009*"van" + 0.008*"displaystyl" + 0.008*"warsaw" + 0.005*"stori" + 0.005*"portrait" + 0.005*"sequenc"'),
 (1,
  '0.021*"warsaw" + 0.016*"tower" + 0.011*"integr" + 0.009*"van" + 0.007*"gogh" + 0.006*"level" + 0.006*"england" + 0.006*"polish"'),
 (2,
  '0.025*"integr" + 0.017*"van" + 0.011*"gogh" + 0.009*"england" + 0.006*"warsaw" + 0.005*"interv" + 0.004*"riemann" + 0.003*"sum"'),
 (3,
  '0.039*"integr" + 0.020*"riemann" + 0.009*"interv" + 0.009*"sum" + 0.008*"partit" + 0.007*"gogh" + 0.007*"tower" + 0.006*"van"'),
 (4,
  '0.018*"integr" + 0.012*"warsaw" + 0.010*"england" + 0.007*"tower" + 0.006*"displaystyl" + 0.005*"riemann" + 0.005*"van" + 0.004*"gogh"')]

In [121]:
# zsumowac wystapienia tematow w corpusie i zobaczyc ktory jest najczestszy

In [122]:
sum_percent = np.zeros(5)
for i in range(len(corpus_sparse)):
    dict_topics = model_corpus.get_document_topics(corpus_sparse[i],minimum_probability=0)
    doc_topics = [topic[1] for topic in dict_topics]
    added = [doc+sum_ for doc,sum_ in zip(doc_topics,sum_percent)]
    sum_percent = added.copy()
    
print(np.argsort(sum_percent)[::-1])


[0 3 1 2 4]


In [123]:
pyLDAvis.enable_notebook()

In [124]:
# topics are sorted from most to least popular! the order of topics is changed
pyLDAvis.gensim.prepare(model_corpus,corpus=corpus_sparse,dictionary=dictionary_sparse)  

/home/kodolamacz/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.038904 -0.004727       1        1  47.599327
1     -0.000010  0.020136       2        1  25.474739
3      0.038872 -0.004937       3        1  18.458681
2     -0.000510 -0.013172       4        1   8.122997
4      0.000552  0.002700       5        1   0.344256, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
1065  Default  275.000000       integr  275.000000  30.0000  30.0000
1865  Default  218.000000       warsaw  218.000000  29.0000  29.0000
779   Default  235.000000      england  235.000000  28.0000  28.0000
1779  Default  132.000000        tower  132.000000  27.0000  27.0000
1547  Default  118.000000      riemann  118.000000  26.0000  26.0000
1830  Default  197.000000          van  197.000000  25.0000  25.0000
709   Default  121.000000  displaystyl  121.000000  24.0000  24.0000
939   Default  226.000000         gogh  226.000000  23.0000  23.0000
1073  Default   65.000000       interv   65.000000  22.0000  22.0000
1718  Default   64.000000          sum   64.000000  21.0000  21.0000
1111  Default   67.000000      kingdom   67.000000  20.0000  20.0000
1161  Default   64.000000       london   64.000000  19.0000  19.0000
1355  Default   54.000000       partit   54.000000  18.0000  18.0000
1147  Default   59.000000        level   59.000000  17.0000  17.0000
469   Default   59.000000      british   59.000000  16.0000  16.0000
1400  Default   74.000000       polish   74.000000  15.0000  15.0000
1612  Default   61.000000      sequenc   61.000000  14.0000  14.0000
671   Default   46.000000        defin   46.000000  13.0000  13.0000
1     Default   42.000000          101   42.000000  12.0000  12.0000
913   Default   44.000000           ft   44.000000  11.0000  11.0000
1127  Default   49.000000      largest   49.000000  10.0000  10.0000
1352  Default   52.000000         park   52.000000   9.0000   9.0000
885   Default   38.000000        floor   38.000000   8.0000   8.0000
699   Default   47.000000   differenti   47.000000   7.0000   7.0000
1347  Default   49.000000        palac   49.000000   6.0000   6.0000
1397  Default   60.000000       poland   60.000000   5.0000   5.0000
526   Default   58.000000       church   58.000000   4.0000   4.0000
1550  Default   50.000000         ring   50.000000   3.0000   3.0000
1693  Default   98.000000        stori   98.000000   2.0000   2.0000
1663  Default   37.000000        south   37.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1663   Topic5    0.183236        south   37.579357   0.3481  -6.0183
581    Topic5    0.148328       comput   29.831532   0.3676  -6.2296
1775   Topic5    0.168343          top   34.530800   0.3479  -6.1030
1073   Topic5    0.291363       interv   65.277542   0.2597  -5.5545
1111   Topic5    0.299121      kingdom   67.483009   0.2528  -5.5282
1127   Topic5    0.227440      largest   49.399399   0.2907  -5.8022
1147   Topic5    0.266695        level   59.659603   0.2612  -5.6429
1718   Topic5    0.282920          sum   64.606102   0.2406  -5.5839
1779   Topic5    0.510731        tower  132.662231   0.1118  -4.9932
469    Topic5    0.252886      british   59.186394   0.2160  -5.6961
500    Topic5    0.117924      central   23.415741   0.3804  -6.4590
779    Topic5    0.739457      england  235.265915  -0.0910  -4.6231
709    Topic5    0.425386  displaystyl  121.558899   0.0164  -5.1760
1352   Topic5    0.216385         park   52.174782   0.1862  -5.8520
699    Topic5    0.198882   differenti   47.541870   0.1949  -5.9363
1547   Topic5    0.386705      riemann  118.896278  -0.0568  -5.2714
1257   Topic5    0.135088        music   28.207939   0.3301  -6.3231
1161   Topic5    0.244255       london   64.588318   0.0940  -5.7308
1502   Topic5    0.167853       region   38.341549   0.2403 